# Modelo de Ia (Knn)

In [35]:
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder
from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.neighbors import KNeighborsClassifier as knn_modelo
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import plotly.express as px
from imblearn.over_sampling import SMOTE

In [36]:
df = pd.read_excel('../base/incluses_tratado.xlsx')


df = df.drop(columns=['Faixa Etária'])

display(df.head(5))

,Participante da Comunidade LGBTQIA+,Identidade de Gênero,Orientação Sexual,Cidade/Estado,Escolaridade,Usa Apps para Oportunidades de Emprego?,Preferência de Cursos,Desafios de Emprego por Gênero,Interesse em Empreender,Situação no Mercado de Trabalho,Usa Redes Sociais?,Finalidade do Uso do App Incluses
0,Sim,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Sim,Prefiro cursos on-line,Sim,Não,Empregado (a),Sim,sim
1,Sim,Mulher cisgênero,Homossexual,Região Sudeste,Ensino Fundamental completo ou cursando,Tenho interesse,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,não
2,Sim,Mulher cisgênero,Pansexual,Região Norte,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Não,Talvez,Empregado (a),Sim,sim
3,Sim,Mulher cisgênero,Bissexual,Região Sudeste,Ensino Fundamental completo ou cursando,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Não,Empregado (a),Sim,não
4,Sim,Homem transgênero,Prefiro não informar,Região Sudeste,Nenhum,Não,"Não tenho preferência, pode ser on-line ou pre...",Sim,Sim,Desempregado (a),Sim,sim


In [37]:
df_resposta = df['Finalidade do Uso do App Incluses']
df_atributo = df.iloc[:, :-1]

#Resposta
label_encoder = LabelEncoder()

#Treino
preprocessador = ColumnTransformer(
    transformers=[
        ('ordinal_encoder', OrdinalEncoder(), make_column_selector(dtype_include=['object','bool'])),
    ],
    remainder='passthrough',
    verbose_feature_names_out= False 
)

# Aplicando o preprocessador nos atributos
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())

In [38]:
# Pré-processamento dos atributos e da resposta
df_atributo = pd.DataFrame(preprocessador.fit_transform(df_atributo), columns=preprocessador.get_feature_names_out())
df_resposta = pd.DataFrame(label_encoder.fit_transform(df_resposta))

smote = SMOTE(random_state=14, k_neighbors=2)
df_atributo, df_resposta = smote.fit_resample(df_atributo, df_resposta)

In [39]:
df_atributos_treino, df_atributos_teste, df_resposta_treino, df_resposta_teste = train_test_split(df_atributo, df_resposta.values, test_size= 0.25000000000000006, random_state=70)

atributos_treino = df_atributos_treino.values
resposta_treino = df_resposta_treino

atributos_teste = df_atributos_teste.values
resposta_teste = df_resposta_teste


# Aplicar SMOTE no conjunto de treino, ajustando o número de vizinhos para evitar o erro
# smote = SMOTE(random_state=12, k_neighbors=1)
# atributos_treino, resposta_treino = smote.fit_resample(df_atributos_treino, df_resposta_treino)

# # Convertendo para arrays
# atributos_teste = df_atributos_teste.values
# resposta_teste = df_resposta_teste.values

In [40]:
numfolds = 5
kf = KFold(n_splits=numfolds, shuffle=True, random_state=0)

k_values = range(1, 31) 
mean_accuracies = []

for k in k_values:
    modelo_knn = knn_modelo(n_neighbors=k)
    validacaoCruzada = cross_val_score(modelo_knn, df_atributo, df_resposta, cv=kf)
    mean_accuracies.append(validacaoCruzada.mean())

fig = px.line(x=k_values, y=mean_accuracies, 
              title='Acurácia em função de k', 
              height=800, width=800)

fig.update_layout(xaxis_title='Valor de k', yaxis_title='Acurácia Média (%)')
fig.show()

best_k = k_values[mean_accuracies.index(max(mean_accuracies))]
print(f'O melhor valor de k é {best_k} com uma acurácia média de {max(mean_accuracies):.2f}')


c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A

O melhor valor de k é 1 com uma acurácia média de 0.85


In [41]:
# Treinamento com o melhor valor de k
classificador_knn = knn_modelo(n_neighbors=best_k)

# Treinando com o conjunto de treino balanceado
classificador_knn.fit(atributos_treino, resposta_treino)

c:\Users\lucalucareli-ieg\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



KNeighborsClassifier(n_neighbors=1)

In [42]:
# Previsões no conjunto de teste
predictions = classificador_knn.predict(atributos_teste)

# Avaliação do modelo
taxa_de_acerto = accuracy_score(resposta_teste, predictions)
print("Taxa de acerto:", round(taxa_de_acerto * 100, 2))

report_classificacao = classification_report(resposta_teste, predictions)
print(report_classificacao)

matriz_de_confusao = confusion_matrix(resposta_teste, predictions)
print("Matriz de Confusão:")
print(matriz_de_confusao)

Taxa de acerto: 100.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00         8

    accuracy                           1.00        21
   macro avg       1.00      1.00      1.00        21
weighted avg       1.00      1.00      1.00        21

Matriz de Confusão:
[[13  0]
 [ 0  8]]
